In [1]:
from datasets import load_dataset

dataset = load_dataset("wikimedia/wikipedia", "20231101.vi")
data = dataset['train']

README.md:   0%|          | 0.00/131k [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/291M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/71.0M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/50.9M [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/316M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1288680 [00:00<?, ? examples/s]

In [2]:
import pandas as pd
df = pd.DataFrame(data)
df = df.iloc[50000:70000]
df

,id,url,title,text
50000,268461,https://vi.wikipedia.org/wiki/Vilafranca%20del...,Vilafranca del Penedès,"Vilafranca del Penedès, đơn giản hơn là Vilafr..."
50001,268462,https://vi.wikipedia.org/wiki/Vilob%C3%AD%20de...,Vilobí del Penedès,Vilobí del Penedès là một đô thị trong ‘‘comar...
50002,268463,https://vi.wikipedia.org/wiki/H%E1%BB%93%20%C3...,Hồ Ông Thoại,"Hồ Ông Thoại, còn có tên Hồ Thoại Sơn, là hồ n..."
50003,268465,https://vi.wikipedia.org/wiki/Raddusa,Raddusa,Raddusa là một đô thị ở tỉnh Catania trong vùn...
50004,268466,https://vi.wikipedia.org/wiki/Gi%E1%BA%A3i%20G...,Giải Grammy cho Nghệ sĩ mới xuất sắc nhất,Nghệ sĩ mới xuất sắc nhất là một trong bốn hạn...
...,...,...,...,...
69995,314188,https://vi.wikipedia.org/wiki/Montjovet,Montjovet,Montjovet là một đô thị tại vùng tự trị thung ...
69996,314190,https://vi.wikipedia.org/wiki/Lamonzie-Montastruc,Lamonzie-Montastruc,"Lamonzie-Montastruc là một xã của Pháp, nằm ở ..."
69997,314191,https://vi.wikipedia.org/wiki/Bionaz,Bionaz,Bionaz là một đô thị tại vùng tự trị thung lũn...
69998,314192,https://vi.wikipedia.org/wiki/Th%E1%BB%95%20c%...,Thổ cẩm Châu Giang,Thổ cẩm Châu Giang là thứ thổ cẩm do người Chă...


In [3]:
from openai import OpenAI
import os
import json

In [4]:
API_KEY = "MY_API_KEY"
client = OpenAI(api_key=API_KEY)

def test(titles):
    formatted_input = "\n".join([f"{i+1}. {title}" for i, title in enumerate(titles)])
    messages = [
        {"role": "system", "content": "You are a helpful assistant that classifies Vietnamese titles as either related to economics or not. Respond with exactly 10 lines of 'True' or 'False', corresponding to the 10 input titles."},
        {"role": "user", "content": formatted_input}
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        store=True,
        messages=messages
    )

    classify_results = response.choices[0].message.content.strip().split("\n")
    
    if len(classify_results) != len(titles):
        raise ValueError("Mismatch between input and output length.")

    return classify_results

In [5]:
counter = 0
final_data = []
batch_size = 10

for i in range(0, len(df), batch_size):
    batch = df.iloc[i:i + batch_size]
    titles = batch['title'].tolist()

    try:
        classify_results = test(titles)
    except Exception as e:
        print(f"Error processing batch starting at index {i}: {e}")
        classify_results = [None] * len(batch)

    for row, classify_result in zip(batch.itertuples(index=False), classify_results):
        final_data.append({
            'id': row.id,
            'url': row.url,
            'title': row.title,
            'text': row.text,
            'classify': classify_result
        })

    counter += len(batch)
    print(f"Processed {counter} rows so far.")

Processed 10 rows so far.
Processed 20 rows so far.
Processed 30 rows so far.
Processed 40 rows so far.
Processed 50 rows so far.
Processed 60 rows so far.
Processed 70 rows so far.
Processed 80 rows so far.
Processed 90 rows so far.
Processed 100 rows so far.
Processed 110 rows so far.
Processed 120 rows so far.
Processed 130 rows so far.
Processed 140 rows so far.
Processed 150 rows so far.
Processed 160 rows so far.
Processed 170 rows so far.
Processed 180 rows so far.
Processed 190 rows so far.
Processed 200 rows so far.
Processed 210 rows so far.
Processed 220 rows so far.
Processed 230 rows so far.
Processed 240 rows so far.
Processed 250 rows so far.
Processed 260 rows so far.
Processed 270 rows so far.
Processed 280 rows so far.
Processed 290 rows so far.
Processed 300 rows so far.
Processed 310 rows so far.
Processed 320 rows so far.
Processed 330 rows so far.
Processed 340 rows so far.
Processed 350 rows so far.
Processed 360 rows so far.
Processed 370 rows so far.
Processed 

In [6]:
final_df = pd.DataFrame(final_data)
final_df.to_csv('/kaggle/working/final_output.csv', index=False)
final_df

,id,url,title,text,classify
0,268461,https://vi.wikipedia.org/wiki/Vilafranca%20del...,Vilafranca del Penedès,"Vilafranca del Penedès, đơn giản hơn là Vilafr...",False
1,268462,https://vi.wikipedia.org/wiki/Vilob%C3%AD%20de...,Vilobí del Penedès,Vilobí del Penedès là một đô thị trong ‘‘comar...,False
2,268463,https://vi.wikipedia.org/wiki/H%E1%BB%93%20%C3...,Hồ Ông Thoại,"Hồ Ông Thoại, còn có tên Hồ Thoại Sơn, là hồ n...",False
3,268465,https://vi.wikipedia.org/wiki/Raddusa,Raddusa,Raddusa là một đô thị ở tỉnh Catania trong vùn...,False
4,268466,https://vi.wikipedia.org/wiki/Gi%E1%BA%A3i%20G...,Giải Grammy cho Nghệ sĩ mới xuất sắc nhất,Nghệ sĩ mới xuất sắc nhất là một trong bốn hạn...,False
...,...,...,...,...,...
19995,314188,https://vi.wikipedia.org/wiki/Montjovet,Montjovet,Montjovet là một đô thị tại vùng tự trị thung ...,False
19996,314190,https://vi.wikipedia.org/wiki/Lamonzie-Montastruc,Lamonzie-Montastruc,"Lamonzie-Montastruc là một xã của Pháp, nằm ở ...",False
19997,314191,https://vi.wikipedia.org/wiki/Bionaz,Bionaz,Bionaz là một đô thị tại vùng tự trị thung lũn...,False
19998,314192,https://vi.wikipedia.org/wiki/Th%E1%BB%95%20c%...,Thổ cẩm Châu Giang,Thổ cẩm Châu Giang là thứ thổ cẩm do người Chă...,False


In [7]:
filtered_df = final_df.copy()
filtered_df['classify'] = filtered_df['classify'].astype(str)
filtered_df['classify'] = filtered_df['classify'].apply(lambda x: x.lower().replace(" ", "") if isinstance(x, str) else x)
filtered_df = filtered_df[filtered_df['classify'] == "true"]
filtered_df.to_csv("/kaggle/working/filtered_output.csv", index=False)
filtered_df

,id,url,title,text,classify
58,268622,https://vi.wikipedia.org/wiki/Tr%C6%B0%E1%BB%9...,Trường Cao đẳng Công chính,Trường Cao đẳng Công Chính là một trường có lị...,true
60,268629,https://vi.wikipedia.org/wiki/T%E1%BA%ADp%20%C...,Tập đoàn Tân Tạo,Tập đoàn Tân Tạo (TANTAO GROUP) bao gồm Công t...,true
202,269127,https://vi.wikipedia.org/wiki/%C4%90%E1%BA%A7u...,Đầu tư tài chính,Đầu tư tài chính là một hình thức đầu tư chủ y...,true
228,269300,https://vi.wikipedia.org/wiki/Ng%C3%A2n%20h%C3...,Ngân hàng thương mại cổ phần,Ngân hàng thương mại cổ phần là cách gọi ở Việ...,true
231,269305,https://vi.wikipedia.org/wiki/B2B,B2B,B2B hoặc BtoB có thể chỉ:\n Business-to -Busin...,true
...,...,...,...,...,...
19568,313102,https://vi.wikipedia.org/wiki/Khai%20hoang,Khai hoang,"Khai hoang là công cuộc mở mang, khai phá ruộn...",true
19569,313104,https://vi.wikipedia.org/wiki/Win-win,Win-win,Win-Win (nguyên tắc thắng-thắng) là một trong ...,true
19596,313227,https://vi.wikipedia.org/wiki/SPSS,SPSS,Thống kê SPSS là một gói phần mềm được sử dụng...,true
19679,313509,https://vi.wikipedia.org/wiki/Bernard%20Madoff,Bernard Madoff,Bernard Lawrence Madoff (29 tháng 4 năm 1938 –...,true
